# US Immigration patterns and their effects on US demographics
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from etl import extract_immigration_data,extract_us_cities_demographics_data,extract_airport_data,extract_countries,extract_entry_ports,extract_travel_modes,extract_us_states,extract_visa_modes, \
transform_immigration_data,transform_us_cities_demographics_data,transform_airport_data,transform_countries_data,transform_entry_ports, \
save_countries_dimension_table,save_entry_ports_dimension_table,save_travel_modes_dimension_table,save_us_states_dimension_table,save_visa_modes_dimension_table,save_airports_dimension_table, \
save_immigration_fact_table,save_us_cities_demographics_dimension_table, validate_data_integrity,get_data_from_parquet_files
from pyspark.sql.functions import col,udf,regexp_replace

### Step 1: Scope the Project and Gather Data

#### Scope of the project
The project focuses finding the patterns of non-immigrant visitors coming to the US each year and their effect on different aspects of the country such as population, tourism, education, etc.. The project solely utilizes Spark framework to do ETL pipeline from various datasources and present some sample data analysis using views created by comparing and combining these datasets.Further work can be done to apply other tools like Airflow for data pipeline orchestration, S3 buckets and Redshift as data storage and data wharehouse,etc.
#### Describe and Gather Data 
3 datasources are used within the project to provide US immigration patterns:
-   I94 Immigration Data: This data comes from [the US National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html). This dataset provide extensive details of the US visitors such as where they came from, which port of entry an city that they came through, and which type of visa they applied for. 
-   U.S. City Demographic Data: This data comes from [OpenSoft](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).This dataset provides a picture of demographics in each state and city.
-   Airport Code Table: This is a simple table of airport codes and corresponding cities, downloaded from [here](https://datahub.io/core/airport-codes#data).

In [2]:
# Read in the i94 data sample
df = pd.read_csv("immigration_data_sample.csv")

In [3]:
df.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.
##### As we read in the data, there are some findings:
- Data may have duplicate values
- Immigrations dataset includes many fields that are not necessary to the project. I decided to use a small set of fields that can be used for our analysis later
- Demographics dataset also has data that are not used by the project
- Airport dataset also has data that are not used by the project
- All datasets involve US states so paritioning by US State code can speed up the querying process


In [4]:
# First create a spark session to read in file data
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.jars","/usr/local/spark/jars/parso-2.0.11.jar,/usr/local/spark/jars/spark-sas7bdat-3.0.0-s_2.11.jar").getOrCreate()

In [5]:
# Read in immigration data
df_immigrations =extract_immigration_data(spark)
df_immigrations.head(5)

[Row(cicid=6.0, i94yr=2016.0, i94mon=4.0, i94cit=692.0, i94res=692.0, i94port='XXX', arrdate=20573.0, i94mode=None, i94addr=None, depdate=None, i94bir=37.0, i94visa=2.0, count=1.0, dtadfile=None, visapost=None, occup=None, entdepa='T', entdepd=None, entdepu='U', matflag=None, biryear=1979.0, dtaddto='10282016', gender=None, insnum=None, airline=None, admnum=1897628485.0, fltno=None, visatype='B2'),
 Row(cicid=7.0, i94yr=2016.0, i94mon=4.0, i94cit=254.0, i94res=276.0, i94port='ATL', arrdate=20551.0, i94mode=1.0, i94addr='AL', depdate=None, i94bir=25.0, i94visa=3.0, count=1.0, dtadfile='20130811', visapost='SEO', occup=None, entdepa='G', entdepd=None, entdepu='Y', matflag=None, biryear=1991.0, dtaddto='D/S', gender='M', insnum=None, airline=None, admnum=3736796330.0, fltno='00296', visatype='F1'),
 Row(cicid=15.0, i94yr=2016.0, i94mon=4.0, i94cit=101.0, i94res=101.0, i94port='WAS', arrdate=20545.0, i94mode=1.0, i94addr='MI', depdate=20691.0, i94bir=55.0, i94visa=2.0, count=1.0, dtadfile=

In [6]:
#Read in US demographics data
df_us_dems = extract_us_cities_demographics_data(spark)
df_us_dems.head(5)

[Row(city='Silver Spring', state='Maryland', median_age=33.8, male_population=40601, female_population=41862, total_population=82463, number_of_veterans=1562, foreign_born=30908, average_household_size=2.6, state_code='MD', race='Hispanic or Latino', count=25924),
 Row(city='Quincy', state='Massachusetts', median_age=41.0, male_population=44129, female_population=49500, total_population=93629, number_of_veterans=4147, foreign_born=32935, average_household_size=2.39, state_code='MA', race='White', count=58723),
 Row(city='Hoover', state='Alabama', median_age=38.5, male_population=38040, female_population=46799, total_population=84839, number_of_veterans=4819, foreign_born=8229, average_household_size=2.58, state_code='AL', race='Asian', count=4759),
 Row(city='Rancho Cucamonga', state='California', median_age=34.5, male_population=88127, female_population=87105, total_population=175232, number_of_veterans=5821, foreign_born=33878, average_household_size=3.18, state_code='CA', race='Blac

In [7]:
#Read in airport code data
df_airports=extract_airport_data(spark)
df_airports.head(5)

[Row(ident='00A', type='heliport', name='Total Rf Heliport', elevation_ft='11', continent='NA', iso_country='US', iso_region='US-PA', municipality='Bensalem', gps_code='00A', iata_code=None, local_code='00A', coordinates='-74.93360137939453, 40.07080078125'),
 Row(ident='00AA', type='small_airport', name='Aero B Ranch Airport', elevation_ft='3435', continent='NA', iso_country='US', iso_region='US-KS', municipality='Leoti', gps_code='00AA', iata_code=None, local_code='00AA', coordinates='-101.473911, 38.704022'),
 Row(ident='00AK', type='small_airport', name='Lowell Field', elevation_ft='450', continent='NA', iso_country='US', iso_region='US-AK', municipality='Anchor Point', gps_code='00AK', iata_code=None, local_code='00AK', coordinates='-151.695999146, 59.94919968'),
 Row(ident='00AL', type='small_airport', name='Epps Airpark', elevation_ft='820', continent='NA', iso_country='US', iso_region='US-AL', municipality='Harvest', gps_code='00AL', iata_code=None, local_code='00AL', coordinat

In [8]:
# extract country data
df_countries=extract_countries(spark)
df_countries.head(5)

[Row(country_code='582', country_name='MEXICO Air Sea, and Not Reported (I-94, no land arrivals)'),
 Row(country_code='236', country_name='AFGHANISTAN'),
 Row(country_code='101', country_name='ALBANIA'),
 Row(country_code='316', country_name='ALGERIA'),
 Row(country_code='102', country_name='ANDORRA')]

In [9]:
# extract port of entries
df_entry_ports = extract_entry_ports(spark)
df_entry_ports.head(5)

[Row(port_code='ALC', port_name='ALCAN, AK'),
 Row(port_code='ANC', port_name='ANCHORAGE, AK'),
 Row(port_code='BAR', port_name='BAKER AAF - BAKER ISLAND, AK'),
 Row(port_code='DAC', port_name='DALTONS CACHE, AK'),
 Row(port_code='PIZ', port_name='DEW STATION PT LAY DEW, AK')]

In [10]:
#extract modes of travel
df_travel_modes=extract_travel_modes(spark)
df_travel_modes.head(5)

[Row(travel_mode_code='1', travel_mode_name='Air'),
 Row(travel_mode_code='2', travel_mode_name='Sea'),
 Row(travel_mode_code='3', travel_mode_name='Land'),
 Row(travel_mode_code='9', travel_mode_name='Not reported')]

In [11]:
#extract us states
df_us_states=extract_us_states(spark)
df_us_states.head(5)

[Row(state_code='AL', state_name='ALABAMA'),
 Row(state_code='AK', state_name='ALASKA'),
 Row(state_code='AZ', state_name='ARIZONA'),
 Row(state_code='AR', state_name='ARKANSAS'),
 Row(state_code='CA', state_name='CALIFORNIA')]

In [12]:
#extract visa types
df_visa_modes=extract_visa_modes(spark)
df_visa_modes.head(5)

[Row(visa_mode_code='1', visa_mode_name='Business'),
 Row(visa_mode_code='2', visa_mode_name='Pleasure'),
 Row(visa_mode_code='3', visa_mode_name='Student')]

In [13]:
# convert code to code and name
#get immigrant info

df_immigrants=df_immigrations.select(col('i94yr').alias('year_of_entry'),col('i94mon').alias('month_of_entry'), \
                                     col('i94cit').alias( 'country_of_origin_code'),col('i94res').alias('country_of_residence_code'), \
                                     col('i94mode').alias('mode_of_entry_code'),col('i94addr').alias('us_address_state_code'), \
                                     col('airline'),col('visatype').alias('visa_type'))
df_immigrants.head(5)

[Row(year_of_entry=2016.0, month_of_entry=4.0, country_of_origin_code=692.0, country_of_residence_code=692.0, mode_of_entry_code=None, us_address_state_code=None, airline=None, visa_type='B2'),
 Row(year_of_entry=2016.0, month_of_entry=4.0, country_of_origin_code=254.0, country_of_residence_code=276.0, mode_of_entry_code=1.0, us_address_state_code='AL', airline=None, visa_type='F1'),
 Row(year_of_entry=2016.0, month_of_entry=4.0, country_of_origin_code=101.0, country_of_residence_code=101.0, mode_of_entry_code=1.0, us_address_state_code='MI', airline='OS', visa_type='B2'),
 Row(year_of_entry=2016.0, month_of_entry=4.0, country_of_origin_code=101.0, country_of_residence_code=101.0, mode_of_entry_code=1.0, us_address_state_code='MA', airline='AA', visa_type='B2'),
 Row(year_of_entry=2016.0, month_of_entry=4.0, country_of_origin_code=101.0, country_of_residence_code=101.0, mode_of_entry_code=1.0, us_address_state_code='MA', airline='AA', visa_type='B2')]

#### Cleaning Steps
Document steps necessary to clean the data

##### Cleanup steps include
- Remove duplicates for all dataframes
- Select enough fields from immigrations, us demographic, and airports datasets for future analysis
- entry_ports with port name divided into city and state
- countries may have Invalid country names. I specified INVALID for all of these records
- airports have airport_code either coming from iata code or if no aita code isnot provided for the airport, use local code if available. Since we focus on US immigration data, we only select airports in the US. Also state code is derived from iso_region

In [14]:
# Only select enough fields from immigration data
df_immigrations_tf=transform_immigration_data(df_immigrations)
df_immigrations_tf.head(5)

[Row(year_of_entry=2016.0, month_of_entry=4.0, country_of_origin_code=104.0, country_of_residence_code=104.0, us_address_state_code='LA', port_of_entry_code='ATL', travel_mode_code=1.0, visa_mode_code=2.0, airline='KL', visa_type='WT'),
 Row(year_of_entry=2016.0, month_of_entry=4.0, country_of_origin_code=107.0, country_of_residence_code=107.0, us_address_state_code='VQ', port_of_entry_code='SAJ', travel_mode_code=1.0, visa_mode_code=2.0, airline='DY', visa_type='B2'),
 Row(year_of_entry=2016.0, month_of_entry=4.0, country_of_origin_code=108.0, country_of_residence_code=108.0, us_address_state_code='NY', port_of_entry_code='RDU', travel_mode_code=1.0, visa_mode_code=2.0, airline='SK', visa_type='WT'),
 Row(year_of_entry=2016.0, month_of_entry=4.0, country_of_origin_code=108.0, country_of_residence_code=108.0, us_address_state_code='TX', port_of_entry_code='WAS', travel_mode_code=1.0, visa_mode_code=1.0, airline='SK', visa_type='WB'),
 Row(year_of_entry=2016.0, month_of_entry=4.0, count

In [15]:
# Only select enough fields from demographics data
df_us_dems_tf=transform_us_cities_demographics_data(df_us_dems)
df_us_dems.head(5)

[Row(city='Silver Spring', state='Maryland', median_age=33.8, male_population=40601, female_population=41862, total_population=82463, number_of_veterans=1562, foreign_born=30908, average_household_size=2.6, state_code='MD', race='Hispanic or Latino', count=25924),
 Row(city='Quincy', state='Massachusetts', median_age=41.0, male_population=44129, female_population=49500, total_population=93629, number_of_veterans=4147, foreign_born=32935, average_household_size=2.39, state_code='MA', race='White', count=58723),
 Row(city='Hoover', state='Alabama', median_age=38.5, male_population=38040, female_population=46799, total_population=84839, number_of_veterans=4819, foreign_born=8229, average_household_size=2.58, state_code='AL', race='Asian', count=4759),
 Row(city='Rancho Cucamonga', state='California', median_age=34.5, male_population=88127, female_population=87105, total_population=175232, number_of_veterans=5821, foreign_born=33878, average_household_size=3.18, state_code='CA', race='Blac

In [16]:
# Only select enough fields from airport data
df_airport_tf=transform_airport_data(df_airports)
df_airport_tf.head(5)

[Row(id='07B', airport_name='Blue Hill Airport', airport_code='07B', state_code='ME'),
 Row(id='09AR', airport_name='Magnolia Hospital Heliport', airport_code='09AR', state_code='AR'),
 Row(id='0LL2', airport_name="Murk's Strip", airport_code='0LL2', state_code='IL'),
 Row(id='0MN8', airport_name='Keller Airport', airport_code='0MN8', state_code='MN'),
 Row(id='0MS4', airport_name="Sheriff's Courthouse Heliport", airport_code='0MS4', state_code='MS')]

In [17]:
# All invalid country names are set to INVALID
df_countries_tf=transform_countries_data(df_countries)
df_countries_tf.head(5)

[Row(country_code='373', country_name='SOUTH AFRICA'),
 Row(country_code='206', country_name='HONG KONG'),
 Row(country_code='214', country_name='MACAU'),
 Row(country_code='394', country_name='INVALID'),
 Row(country_code='589', country_name='INVALID')]

In [18]:
# entry_ports with port name divided into city and state
df_entry_ports_tf=transform_entry_ports(df_entry_ports)
df_entry_ports_tf.head(5)

[Row(port_code='HNN', port_name='HANNAH, ND', city_name='HANNAH', state_code='ND'),
 Row(port_code='PKC', port_name='POKER CREEK, AK', city_name='POKER CREEK', state_code='AK'),
 Row(port_code='POM', port_name='PORTLAND, ME', city_name='PORTLAND', state_code='ME'),
 Row(port_code='PFN', port_name='INVALID', city_name=None, state_code=None),
 Row(port_code='NIK', port_name='NIKISKI, AK', city_name='NIKISKI', state_code='AK')]

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
STAR schema is used with immigrations as fact table and others are dimension ones.
![Data Models](capstone_data_models.png)
#### 3.2 Mapping Out Data Pipelines
Data are extracted from different datasourced (mentioned above), transformed into meaningful data models, and then saved as parquet files.



### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Data models come from data extraction and transformation using spark dataframe, then saved into parquet files. Some notes:
- As state code are part of many data models, it will be used as partition for them to speed up the query process.
- Secondary paritioning with city is used for dim_entry_ports and dim_us_dems as they are related 

In [19]:
# Write code here
save_countries_dimension_table(df_countries)
save_entry_ports_dimension_table(df_entry_ports_tf)
save_travel_modes_dimension_table(df_travel_modes)
save_us_states_dimension_table(df_us_states)
save_visa_modes_dimension_table(df_visa_modes)
save_airports_dimension_table(df_airport_tf)
save_immigration_fact_table(spark,df_immigrations_tf,df_countries_tf,df_us_states,df_entry_ports_tf,df_travel_modes,df_visa_modes)
save_us_cities_demographics_dimension_table(df_us_dems_tf)

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. 
- Data are validated to make sure they are not empty
- Fact table has valid data that are backed by dimension tables' ones.
 
Run Quality Checks

In [20]:
# Perform quality checks here
validate_data_integrity(spark)

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

`fact_immigrations`: data come from I94 immigration dataset
```
- year_of_entry: the year of the arrival
- month_of_entry: the month of the arrival
- country_of_origin_code: the code of the visitor's country of origin
- country_of_residence_code: the code of the visitor's country of residence
- us_address_state_code: the code for the US state where the port of entry is located
- port_of_entry_code; the code for the port of entry
- travel_mode_code: specifies which means of transportaion the visitor uses to travel to the US, e.g. air,sea
- visa_mode_code: type of visa category, e.g. student or tourist
- airline: if the visitor travels by air, which airline they use
- visa_type: type of the visa, e.g B1/B2 for tourist, F1 for student
```

`dim_us_dems`: the demographics for US cities. These data come from the US demographics dataset mentioned above
```
 - state_code: the state where the city is located
 - city: the city's name
 - total_population: the population for the city
 - foreign_born: the population of the foregin born people
```
 
`dim_airports`: the US' airport data. These data come from the Airport dataset mentioned above
```
 - id: the id that distinguish the airport
 - aiport_name: the name of the airport
 - state_code: the state where the airport is located
 - airport_code; this is either IATA code or the local code
```
 
 
`dim_countries, dim_entry_ports,dim_us_states,dim_travel_modes,dim_visa_modes`: the data are parsed from I94 label metadata file and provide the mapping for the codes from the fact_immigrations into their meaningful name. 

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

### Tools and technologies
- Docker and docker-compose are used to provide immutable deployment of development environment at my local workspace. The image `jupyter/all-spark-notebook:spark-2` is used to provide spark 2 support.
- To read in SAS files using spark, `spark-sas7bdat` jar is used alongside with its dependency: `parso`
- `pyspark` is used for the whole ETL process as it provides paralled and high througput processing. 
- `parquet` is used as data model storage format for its small storage size and high loading/saving performance

### Approaches for other scenarios not included in the project 
- The data was increased by 100x: Spark is good for data processing: Highly available and scalable Spark cluster can be used to support increase in data processing
- The data populates a dashboard that must be updated on a daily basis by 7am every day: Data pipeline using orchestration tools like Airflow or Jenkins can provide scheduling
- The database needed to be accessed by 100+ people: Datawarehousing tools like AWS Redshift which supports cluster and partitioning/sharding can be used to provide parrallel processing.Read replicas can be added for faster retrieval.

### Sample data analysis
- Find the top countries of residence of the visitors
- Find the top states that received the most visitors
- Find top states for international students and its foreign-born population to find if there are any correlations
     * We can see that states that have high foreign-born population will also have high international students. Exception some states with low population like MA but has many academic institutions
- Find the busiest airports in the US as international destinations

In [21]:
#top 10 ports that receive the most visistors
# load data from parquet files
df_airports_load, df_countries_load, df_entry_ports_load, \
    df_immigrations_load, df_travel_modes_load, df_us_dems_load, \
    df_us_states_load, df_visa_modes_load \
        = get_data_from_parquet_files(spark)


In [22]:
# create temp views
df_immigrations_load.createOrReplaceTempView('immigration_temp_view')
df_countries_load.createOrReplaceTempView('countries_temp_view')
df_us_states_load.createOrReplaceTempView('us_state_temp_view')
df_entry_ports_load.createOrReplaceTempView('ports_temp_view')
df_travel_modes_load.createOrReplaceTempView('travel_modes_temp_view')
df_visa_modes_load.createOrReplaceTempView('visa_modes_temp_view')
df_airports_load.createOrReplaceTempView('airports_temp_view')
df_us_dems_load.createOrReplaceTempView('us_dems_temp_view')

In [23]:
# Find the top countries of residence of the visitors
spark.sql("""
    SELECT country_name,count(*) as number_of_visitors
    FROM immigration_temp_view itv,countries_temp_view ctv
    WHERE itv.country_of_residence_code = ctv.country_code
    GROUP BY country_name
    ORDER BY number_of_visitors DESC
    
""").show()

+--------------------+------------------+
|        country_name|number_of_visitors|
+--------------------+------------------+
|      UNITED KINGDOM|             21999|
|          CHINA, PRC|             14279|
|              FRANCE|             12365|
|               INDIA|             12210|
|MEXICO Air Sea, a...|             11335|
|               JAPAN|             11177|
|           AUSTRALIA|              9405|
|              BRAZIL|              8673|
|               ITALY|              7253|
|               SPAIN|              7095|
|         SWITZERLAND|              7082|
|         NETHERLANDS|              7002|
|              ISRAEL|              5014|
|            COLOMBIA|              4768|
|              TAIWAN|              4680|
|           ARGENTINA|              4552|
|              SWEDEN|              4393|
|             BELGIUM|              4331|
|             IRELAND|              4150|
|             GERMANY|              3892|
+--------------------+------------

In [24]:
# find the top states that received the most visitors
spark.sql("""
    SELECT  us_address_state_code, count(*) AS number_of_international_students
        FROM immigration_temp_view itv,visa_modes_temp_view vmtv
        WHERE itv.visa_mode_code=vmtv.visa_mode_code AND
              vmtv.visa_mode_name='Student'
        GROUP BY us_address_state_code
        ORDER BY number_of_international_students DESC
        
""").show()

+---------------------+--------------------------------+
|us_address_state_code|number_of_international_students|
+---------------------+--------------------------------+
|                   CA|                            2706|
|                   NY|                            2116|
|                   FL|                            1663|
|                   MA|                            1208|
|                   TX|                             781|
|                   IL|                             675|
|                   PA|                             496|
|                   WA|                             435|
|                   MI|                             393|
|                   NJ|                             358|
|                   VA|                             336|
|                   GA|                             271|
|                   OH|                             268|
|                   UT|                             259|
|                   RI|        

In [25]:
# Find top states for international students and its foreign-born population
# We can see that states that have high foreign-born population will also have high international students
# Exception some states with low population like MA but has many academic institutions
spark.sql("""
    SELECT * FROM 
    (
        SELECT  us_address_state_code, count(*) AS number_of_visitors
        FROM immigration_temp_view itv,visa_modes_temp_view vmtv
        WHERE itv.visa_mode_code=vmtv.visa_mode_code AND
              vmtv.visa_mode_name='Student'
        GROUP BY us_address_state_code
        ORDER BY number_of_visitors DESC
    ) itvs
    LEFT JOIN 
    (
        SELECT state_code, sum(foreign_born) AS number_of_foreign_born
        FROM us_dems_temp_view udtv
        GROUP BY state_code
        ORDER BY number_of_foreign_born DESC
    ) udtvs
    ON itvs.us_address_state_code=udtvs.state_code
""").show()

+---------------------+------------------+----------+----------------------+
|us_address_state_code|number_of_visitors|state_code|number_of_foreign_born|
+---------------------+------------------+----------+----------------------+
|                   CA|              2706|        CA|               7448257|
|                   NY|              2116|        NY|               3438081|
|                   FL|              1663|        FL|               1688931|
|                   MA|              1208|        MA|                520597|
|                   TX|               781|        TX|               2942164|
|                   IL|               675|        IL|                941735|
|                   PA|               496|        PA|                290860|
|                   WA|               435|        WA|                440962|
|                   MI|               393|        MI|                249772|
|                   NJ|               358|        NJ|                477028|

In [26]:
# Find the busiest airports
spark.sql("""
    SELECT airport_name, count(*) AS number_of_visitors
    FROM immigration_temp_view itv
    JOIN travel_modes_temp_view tmtv
    ON itv.travel_mode_code = tmtv.travel_mode_code
    JOIN airports_temp_view atv
    ON itv.port_of_entry_code=atv.airport_code
    WHERE tmtv.travel_mode_name='Air'
    GROUP BY airport_name
    ORDER BY number_of_visitors DESC
""").show()

+--------------------+------------------+
|        airport_name|number_of_visitors|
+--------------------+------------------+
|Miami Internation...|             19223|
|San Fernando Airport|             12199|
|   Lakefront Airport|             11998|
|William P Hobby A...|             11034|
|Hartsfield Jackso...|             10977|
|Orlando Executive...|              9861|
|   Dallas Love Field|              8417|
|General Edward La...|              8343|
|Seattle Tacoma In...|              6930|
|  Point Hope Airport|              6514|
|Torrington Munici...|              5742|
|Coleman A. Young ...|              5011|
|Denver Internatio...|              3952|
|Palm Springs Inte...|              3751|
|          Page Field|              3744|
|Charlotte Douglas...|              3095|
|     Kahului Airport|              2831|
|Southern Californ...|              2459|
|Dubois Municipal ...|              2079|
|  Sand Point Airport|              1539|
+--------------------+------------